<a href="https://colab.research.google.com/github/maleehahassan/HIDA_Into_to_DL/blob/main/01_networks_as_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Overview: Penguin Species Classification with Neural Networks

This notebook demonstrates how to build a neural network to classify penguin species using the Palmer Penguins dataset. The workflow includes:

- **Data Loading & Exploration**: The dataset is loaded and visualized to understand relationships between features and species.
- **Preprocessing**: Irrelevant columns are dropped, missing values are removed, and the target variable ('species') is one-hot encoded for classification.
- **Data Splitting**: The data is split into training and test sets, stratified by species to maintain class balance.
- **Model Building**: A simple feedforward neural network is constructed using TensorFlow/Keras, with one hidden layer and a softmax output for multiclass classification.
- **Training**: The model is trained for 100 epochs, and training loss is visualized to monitor learning progress.
- **Prediction & Evaluation**: Predictions are made on the test set, and the model's performance is evaluated using a confusion matrix and heatmap to show classification accuracy for each species.

This notebook guides learners through the process of preparing data, building and training a neural network for multiclass classification, and evaluating results using visual tools.

In [ ]:
# Import seaborn for data visualization
import seaborn as sns

In [ ]:
# Load the penguins dataset from seaborn
penguins = sns.load_dataset('penguins')

In [ ]:
# Visualize pairwise relationships in the dataset, colored by species
sns.pairplot(penguins, hue="species")

In [ ]:
# Convert the 'species' column to a categorical data type
penguins['species'] = penguins['species'].astype('category')

In [ ]:
# Drop two columns and the rows that have NaN values in them
# Drop the 'island' and 'sex' columns, then remove rows with missing values
penguins_filtered = penguins.drop(columns=['island', 'sex']).dropna()

# Extract columns corresponding to features
# Extract feature columns (all except 'species') for model input
penguins_features = penguins_filtered.drop(columns=['species'])


In [ ]:
# Import pandas for data manipulation
import pandas as pd

# Convert the target variable 'species' to one-hot encoded format
target = pd.get_dummies(penguins_filtered['species'])

# Display the first 5 rows of the one-hot encoded target
target.head() # print out the top 5 to see what it looks like.

In [ ]:
# Import train_test_split to split data into training and test sets
from sklearn.model_selection import train_test_split

# Split features and target into training and test sets, stratified by species
X_train, X_test, y_train, y_test = train_test_split(penguins_features, target,test_size=0.2, random_state=0, shuffle=True, stratify=target)


In [ ]:
# Print the count of each species in the full target set
print(target.value_counts())

In [ ]:
# Print the count of each species in the training set
print(y_train.value_counts())

In [ ]:
# Print the count of each species in the test set
print(y_test.value_counts())

In [ ]:
# Import keras for building neural networks
from tensorflow import keras

In [ ]:
# Set random seeds for reproducibility
from numpy.random import seed
seed(1)
from tensorflow.random import set_seed
set_seed(2)


In [ ]:
# Define the input layer with the shape of the feature set
inputs = keras.Input(shape=(X_train.shape[1],))


In [ ]:
# Add a hidden layer with 10 units and ReLU activation
hidden_layer = keras.layers.Dense(10, activation="relu")(inputs)

In [ ]:
# Add an output layer with 3 units (one for each species) and softmax activation
output_layer = keras.layers.Dense(3, activation="softmax")(hidden_layer)

In [ ]:
# Build the model by specifying the input and output layers
model = keras.Model(inputs=inputs, outputs=output_layer)

# Display the model architecture
model.summary()


In [ ]:
# Compile the model with Adam optimizer and categorical crossentropy loss
model.compile(optimizer='adam', loss=keras.losses.CategoricalCrossentropy())

In [ ]:
# Train the model on the training set for 100 epochs
history = model.fit(X_train, y_train, epochs=100)

In [ ]:
# Plot the training loss over epochs
sns.lineplot(x=history.epoch, y=history.history['loss'])

In [ ]:
# Make predictions on the test set
y_pred = model.predict(X_test)

# Convert predictions to a DataFrame for easier analysis
prediction = pd.DataFrame(y_pred, columns=target.columns)
prediction


In [ ]:
# Get the species with the highest predicted probability for each observation
predicted_species = prediction.idxmax(axis="columns")
predicted_species


In [ ]:
# Import confusion_matrix to evaluate model performance
from sklearn.metrics import confusion_matrix

# Get the true species labels for the test set
true_species = y_test.idxmax(axis="columns")

# Compute the confusion matrix
matrix = confusion_matrix(true_species, predicted_species)
print(matrix)


In [ ]:
# Convert to a pandas dataframe
confusion_df = pd.DataFrame(matrix, index=y_test.columns.values, columns=y_test.columns.values)

# Set the names of the x and y axis, this helps with the readability of the heatmap.
confusion_df.index.name = 'True Label'
confusion_df.columns.name = 'Predicted Label'


In [ ]:
# Visualize the confusion matrix as a heatmap
sns.heatmap(confusion_df, annot=True)